# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [3]:
value_date = Date(2, 8, 2007)
settle_date = value_date.add_weekdays(1)

## Build Ibor Curve

In [4]:
dcType = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(value_date, "1D", 0.0500, dcType); depos.append(depo)
depo = IborDeposit(settle_date, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settle_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
step_in_date = value_date.add_weekdays(1)

In [6]:
maturity3Y = value_date.next_cds_date(36)
maturity5Y = value_date.next_cds_date(60)
maturity7Y = value_date.next_cds_date(84)
maturity10Y = value_date.next_cds_date(120)

### Heterogeneous Curves

In [7]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_date, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [8]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40 

In [9]:
cdsIndex = CDSIndexPortfolio()

In [10]:
spd3Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(value_date, step_in_date, maturity10Y, heteroIssuerCurves)

In [11]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678793982955575
Homogeneous curve 5Y: 35.53914238947779
Homogeneous curve 7Y: 49.01183816666461
Homogeneous curve 10Y: 61.41376931517749


In [12]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_date, maturity3Y, spd3Y)
    cds5Y = CDS(step_in_date, maturity5Y, spd5Y)
    cds7Y = CDS(step_in_date, maturity7Y, spd7Y)
    cds10Y = CDS(step_in_date, maturity10Y, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_date, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Tranches

In [13]:
trancheMaturity = maturity5Y
tranche1 = CDSTranche(value_date, trancheMaturity, 0.00, 0.03)
tranche2 = CDSTranche(value_date, trancheMaturity, 0.03, 0.06)
tranche3 = CDSTranche(value_date, trancheMaturity, 0.06, 0.09)
tranche4 = CDSTranche(value_date, trancheMaturity, 0.09, 0.12)
tranche5 = CDSTranche(value_date, trancheMaturity, 0.12, 0.22)
tranche6 = CDSTranche(value_date, trancheMaturity, 0.22, 0.60)
tranche7 = CDSTranche(value_date, trancheMaturity, 0.00, 0.60)
tranche8 = CDSTranche(value_date, trancheMaturity, 0.00, 1.00)

In [14]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [15]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [16]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))
for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_date,homoIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, num_points, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.088146
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.088146
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   904.030205
                    FinLossDistributionBuilder.LHP    50   0.00000   3.00000   914.366815
              FinLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.580083
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.580083
               FinLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   215.722567
                    FinLossDistributionBuilder.LHP    50   3.00000   6.00000   226.808804
              FinLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.124297
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   9.00000   102.124297
         

## Heterogeneous Portfolio Results

In [17]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))

for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_date,heteroIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, num_points, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.541331
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.473315
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   980.861757
                    FinLossDistributionBuilder.LHP    50    0.00000   3.00000   914.736913
              FinLossDistributionBuilder.RECURSION    50    3.00000   6.00000   230.886128
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   230.978141
               FinLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   202.216426
                    FinLossDistributionBuilder.LHP    50    3.00000   6.00000   226.808075
              FinLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.195509
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    6.00000   9.00000    87.253318

Copyright (c) 2020 Dominic O'Kane